# Geometry boundary to X3D Nurbs

In [1]:
import xml.etree.ElementTree as ET

from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_NurbsConvert
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_BSplineSurface

from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Display.WebGl.jupyter_renderer import create_download_link
import OCC.Extend.DataExchange.x3d_standard.x3d as XX3D

from IPython.display import HTML

x3d.py package loaded, have fun with X3D Graphics!


In [2]:
from OCC.Extend.DataExchange.STEP import read_step_file
# courtesy https://grabcad.com/library/aeroplane-65/details?folder_id=4438014
base_shape = read_step_file('./plane.stp')

In [3]:
# conversion to a nurbs representation
converted_shape = BRepBuilderAPI_NurbsConvert(base_shape, True).Shape()
# now, all edges should be BSpline curves and surfaces BSpline surfaces

In [4]:
# now traverse all shapes and map to a Shape with NurbPatch nodes
# https://castle-engine.io/x3d_implementation_nurbs.php#section_homogeneous_coordinates
expl = TopologyExplorer(converted_shape)

face_idx = 1

all_shape_nodes = []

# the material for this face
face_mesh_material = XX3D.Material(diffuseColor=(0.4,.4,0.4),
                                   specularColor=(0.9, 0.9, 0.9),
                                   shininess=1, ambientIntensity=0.1)

for face in expl.faces():
    surf = BRepAdaptor_Surface(face, True)
    surf_type = surf.GetType()
    # check each of the is a BSpline surface
    # it should be, since we used the nurbs converter before
    if not surf_type == GeomAbs_BSplineSurface:
        print("the face was not converted to a GeomAbs_BSplineSurface")
        continue
        
    # get the nurbs
    bsrf = surf.BSpline()
    # x3d does not have periodic nurbs
    bsrf.SetUNotPeriodic()
    bsrf.SetVNotPeriodic()
    # bspline surface properties
    # order = degree + 1
    u_order = bsrf.UDegree() + 1
    v_order = bsrf.VDegree() + 1

    nb_u_poles = bsrf.NbUPoles()
    nb_u_knots = bsrf.NbUKnots()

    nb_v_poles = bsrf.NbVPoles()
    nb_v_knots = bsrf.NbVKnots()
    
    # fill in the x3d template with nurbs information
    
    nurbs_patch_surface_node = XX3D.NurbsPatchSurface()
    nurbs_patch_surface_node.solid = False
    nurbs_patch_surface_node.DEF = 'nurbs_%i' % face_idx

    nurbs_patch_surface_node.vClosed = bsrf.IsVClosed()
    nurbs_patch_surface_node.uClosed = bsrf.IsUClosed()

    nurbs_patch_surface_node.uOrder = u_order
    nurbs_patch_surface_node.vOrder = v_order

    # the uTesselation and vTesselation fields
    # are set to be equal to nbr of knots
    # it should be discussed
    nurbs_patch_surface_node.uTessellation = nb_u_knots
    nurbs_patch_surface_node.vTessellation = nb_v_knots

    nurbs_patch_surface_node.uDimension = nb_u_poles
    nurbs_patch_surface_node.vDimension = nb_v_poles
    
    # knots vector
    u_knot = []
    for iu in range(nb_u_knots):
        mu = bsrf.UMultiplicity(iu + 1) # repeat u knots as necessary
        u_knot.extend([bsrf.UKnot(iu + 1)] * mu)

    nurbs_patch_surface_node.uKnot = u_knot
    
    v_knot = []  
    for iv in range(nb_v_knots):
        mv = bsrf.VMultiplicity(iv + 1) # repeat v knots as necessary
        v_knot.extend([bsrf.VKnot(iv + 1)] * mv)
    
    nurbs_patch_surface_node.vKnot = v_knot
    
    # weights can be None
    weights = []
    if bsrf.Weights() is not None:
        for iw in range(nb_v_poles):
            for jw in range(nb_u_poles):
                weights.append(bsrf.Weight(jw + 1, iw + 1))
        nurbs_patch_surface_node.weight = weights

    # the control points
    control_points_coord_node = XX3D.Coordinate()
    control_points_coord_node.containerField = 'controlPoint'
    control_point_coordinates = []
    # control points (aka poles), as a 2d array
    if bsrf.Poles() is not None:
        for ip in range(nb_v_poles):
            for jp in range(nb_u_poles):
                p = bsrf.Pole(jp + 1, ip + 1)
                # note: x3d need preweighted control points
                # see https://www.opencascade.com/doc/occt-7.4.0/refman/html/class_b_rep_builder_a_p_i___nurbs_convert.html#details
                w = bsrf.Weight(jp + 1, ip + 1)
                p_x = p.X() * w
                p_y = p.Y() * w
                p_z = p.Z() * w
                control_point_coordinates.append([p_x, p_y, p_z])
    control_points_coord_node.point = control_point_coordinates
    nurbs_patch_surface_node.controlPoint = control_points_coord_node
    
    # create a new shape node
    shape_node = XX3D.Shape()
    shape_node.geometry = nurbs_patch_surface_node
    shape_node.appearance = XX3D.Appearance(material = face_mesh_material)
    # and add this shape to the list of shapes
    all_shape_nodes.append(shape_node)
    
    face_idx += 1

In [5]:
# complete x3d file
x3dscene = XX3D.Scene(children=all_shape_nodes)
x3ddoc = XX3D.X3D(Scene=x3dscene)

# override the containerField content
xml_content = x3ddoc.XML()
# root node
x3d_element = list(ET.XML(xml_content).iter('X3D'))[0]

# weird, x3d.py does not allow to override the containerField property
for c in x3d_element.iter('Coordinate'):
    c.set('containerField', 'controlPoint')
# back to XML
new_xml = ET.tostring(x3d_element, encoding="unicode", short_empty_elements=False)

In [6]:
create_download_link(a_str=new_xml, filename="plane_nurbs.x3d")

HTML(value='<a download="plane_nurbs.x3d" href="data:text/x3d;base64,PFgzRCB4bWxuczp4c2k9Imh0dHA6Ly93d3cudzMub…

In [7]:
# create x3dom html
_X3DOM_HEADER = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script>
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>
'''
x3d_element = list(ET.XML(new_xml).iter('X3D'))[0]

# remove gama correction
next(x3d_element.iter('Scene')).append(ET.XML('<Environment gammaCorrectionDefault="none"/>'))

# add crease angle to IndexedTriangleSet instances, supported by x3dom
# but not part of the X3D standard
for idx_ts in x3d_element.iter('IndexedTriangleSet'):
    idx_ts.set('creaseAngle', '0.2')

x3dHTML = ET.tostring(x3d_element, encoding="unicode", short_empty_elements=False)
x3dHTML = x3dHTML.replace("visible=", 'render=')
x3dom_html = _X3DOM_HEADER + x3dHTML

In [ ]:
HTML(x3dom_html)